In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

In [6]:
pip install transformers[torch]


Note: you may need to restart the kernel to use updated packages.


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
x_train,x_test,y_train,y_test=train_test_split(X_labeled,y_labeled,test_size=0.3,random_state=42)

In [9]:
classifier.fit(x_train,y_train)

RandomForestClassifier(random_state=42)

In [10]:
y_pred=classifier.predict(x_test)

In [11]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.80      1.00      0.89         4

    accuracy                           0.80         5
   macro avg       0.40      0.50      0.44         5
weighted avg       0.64      0.80      0.71         5



C:\python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## accuracy 

In [12]:
from sklearn.model_selection  import cross_val_score
scores=cross_val_score(classifier,X_labeled,y_labeled,cv=5,scoring='accuracy')
print(f'Cross-validation Accuracy score:{scores}')

Cross-validation Accuracy score:[0.66666667 0.66666667 0.66666667 0.66666667 0.66666667]


In [13]:
print(f"mean Accuracy :{scores.mean():.2f}")

mean Accuracy :0.67


In [2]:
#using plagarism


In [14]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
import pdfplumber
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize NLTK tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Directory containing all PDFs
PDF_DIR = "papers"  # Replace with the path to your dataset folder

# Define labeled (15) and unlabeled (135) PDFs
labeled_pdfs = [
    "P001.pdf", "P002.pdf", "P003.pdf", "P004.pdf", "P005.pdf", "P006.pdf",
    "P007.pdf", "P008.pdf", "P009.pdf", "P010.pdf", "P011.pdf", "P012.pdf",
    "P013.pdf", "P014.pdf", "P015.pdf"
]
unlabeled_pdfs = [pdf for pdf in os.listdir(PDF_DIR) if pdf not in labeled_pdfs]

# Preprocessing function for text
def preprocess_text(text):
    text = text.lower().replace("\n","")
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = [word for word in text.split() if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Function to extract sections from a PDF
def extract_sections(pdf_path):
    sections = {'Abstract': '', 'Methodology': '', 'Results': '', 'Conclusion': ''}
    section_keywords = ['Abstract', 'Methodology', 'Results', 'Conclusion']
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue
            for keyword in section_keywords:
                if keyword.lower() in text.lower():
                    sections[keyword] += text
    for key in sections:
        if not sections:
            if not sections[key]:
                sections[key]=" ".join(page.extract_text() for page in pdf.pages if page.extract_text())
    return sections

# Extract features from PDFs
def extract_features(pdfs, pdf_dir):
    data = []
    for pdf in pdfs:
        pdf_path = os.path.join(pdf_dir, pdf)
        sections = extract_sections(pdf_path)
        features = {
            'filename': pdf,
            'abstract': preprocess_text(sections.get('Abstract', '')),
            'methodology': preprocess_text(sections.get('Methodology', '')),
            'results': preprocess_text(sections.get('Results', '')),
            'conclusion': preprocess_text(sections.get('Conclusion', ''))
        }
        data.append(features)
    return pd.DataFrame(data)

# Extract features for labeled and unlabeled data
labeled_data = extract_features(labeled_pdfs, PDF_DIR)
unlabeled_data = extract_features(unlabeled_pdfs, PDF_DIR)

# Load known labels for the labeled data
# Replace this with your actual labels: 0 = Non-Publishable, 1 = Publishable
labeled_data['label'] = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # Example labels

# Combine text fields for vectorization
labeled_data['combined_text'] = labeled_data['abstract'] + labeled_data['methodology'] + labeled_data['results'] + labeled_data['conclusion']
unlabeled_data['combined_text'] = unlabeled_data['abstract'] + unlabeled_data['methodology'] + unlabeled_data['results'] + unlabeled_data['conclusion']

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(max_features=500)
X_labeled = vectorizer.fit_transform(labeled_data['combined_text']).toarray()
y_labeled = labeled_data['label']

X_unlabeled = vectorizer.transform(unlabeled_data['combined_text']).toarray()

# Check plagiarism using cosine similarity
similarity_threshold = 0.8  # Adjust threshold as needed
unlabeled_data['plagiarism_flag'] = 0

for i, unlabeled_vector in enumerate(X_unlabeled):
    similarities = cosine_similarity([unlabeled_vector], X_labeled)
    if similarities.max() > similarity_threshold:
        unlabeled_data.loc[i, 'plagiarism_flag'] = 1

# Train classifier
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_labeled, y_labeled)

# Predict on the remaining 135 papers
unlabeled_data['predicted_label'] = classifier.predict(X_unlabeled)

# Combine plagiarism flag with predicted label
unlabeled_data['final_label'] = unlabeled_data.apply(
    lambda row: 'Non-Publishable' if row['plagiarism_flag'] == 1 else ('Publishable' if row['predicted_label'] == 1 else 'Non-Publishable'),
    axis=1
)

# Save predictions to a CSV
unlabeled_data[['filename', 'final_label']].to_csv('final2_labels.csv', index=False)
print("Predictions saved to 'final2_labels.csv'.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\patet\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Predictions saved to 'final2_labels.csv'.


In [4]:
#check the pdf is published or not

In [15]:
def check_new_pdf(pdf_path, vectorizer, classifier, labeled_data, similarity_threshold=0.8):
    # Extract and preprocess the new PDF
    sections = extract_sections(pdf_path)
    combined_text = preprocess_text(sections.get('Abstract', '')) + \
                    preprocess_text(sections.get('Methodology', '')) + \
                    preprocess_text(sections.get('Results', '')) + \
                    preprocess_text(sections.get('Conclusion', ''))
    
    # Vectorize the new PDF's text
    new_vector = vectorizer.transform([combined_text]).toarray()
    
    # Check for plagiarism
    labeled_vectors = vectorizer.transform(labeled_data['combined_text']).toarray()
    similarities = cosine_similarity(new_vector, labeled_vectors)
    if similarities.max() > similarity_threshold:
        return "Non-Publishable (Plagiarized)"
    
    # Predict using the trained classifier
    predicted_label = classifier.predict(new_vector)[0]
    return "Publishable" if predicted_label == 1 else "Non-Publishable"

# Example usage
new_pdf_path = "DEEP_LEARNING.pdf"  # Replace with the path to your new PDF
result = check_new_pdf(new_pdf_path, vectorizer, classifier, labeled_data)
print(f"The new PDF is classified as: {result}")


The new PDF is classified as: Publishable
